## Mace MD Demo notebook

This should demonstrate the bare bones of the API for setting up a variety of MD jobs with hybrid MACE-classical hamiltonians.  The same functionalty is exposed through the `mace-md` command line program that is installed with the `mace-openmm-interop` package


Your vs code server instance should have an attached CUDA GPU for this to run!

In [1]:
from mace_openmm.mixed_system import MixedSystem
import torch
import logging

/rds/user/jhm72/hpc-work/openmmtools/openmmtools/storage/iodrivers.py:88: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  return '/'.join([path_part.strip('/ ') for path_part in split_path if path_part is not ''])
/rds/user/jhm72/hpc-work/openmmtools/openmmtools/storage/iodrivers.py:703: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self._output_mode is 'a':


In [2]:
logger = logging.getLogger("INFO")

In [3]:
# Change inputs in this cell only
file = "../tests/example_data/5n.sdf" # path to the system to be simulated, can be either protein ligand complex or isolated (sdf)
ml_mol = file # path to the proper topology of the small molecule to be modeleld by mace, since pdb files are not sufficiently detailed.  this should be sdf. 
model_path = "../tests/example_data/MACE_SPICE_larger.model"
forcefields = ["amber/protein.ff14SB.xml", "amber/tip3p_standard.xml","amber14/DNA.OL15.xml"]
# name of the residue to be parametrised by mace, should correspond to whatever it is called in the ml_mol sdf file and system file
resname = "5n"
padding = 1.2
ionicStrength = 0.15
nonbondedCutoff = 1.0
potential = "mace"
temperature = 298.15
dtype = torch.float64
# n2 impl is more reliable
neighbour_list = "torch_nl_n2"

output_dir = "./output"

# controls whether the system will be solvated with only the specified portion in ml_mol parametrised by mace
pure_ml_system = False


In [5]:
mixedSystem = MixedSystem(file=file,
                          ml_mol=ml_mol,
                          model_path=model_path,
                          forcefields=forcefields,
                          resname=resname,
                          padding=padding,
                          ionicStrength=ionicStrength,
                          nonbondedCutoff=nonbondedCutoff,
                          potential=potential,
                          temperature=temperature,
                          dtype=dtype,
                          neighbour_list=neighbour_list,
                          output_dir=output_dir,
                          pure_ml_system=pure_ml_system
                          )

In [ ]:
# runs MD with the mixed hamiltonian

mixedSystem.run_mixed_md(steps=1000, interval=100, output_file="./output_md_test.pdb")

In [ ]:
# runs Markov chain monte carlo replica exchange to interpoolate between MM and MM/ML descriptions of the system
mixedSystem.run_replex_equilibrium_fep(replicas=10, restart=False, steps=1000)